In [1]:
print("hello world")

hello world


In [9]:
import os
import re
import json
import numpy as np
import requests
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from tqdm import tqdm

# ==================================================
# === Config ===
API_ENDPOINT = 'http://localhost:400' + str(2) + '/simulation' # NOT THE LS VERSION
HEADERS = {"Content-Type": "application/json"}

# ==================================================
# === Utility functions ===

def parse_filename(filename):
    """
    Parses an image filename to extract:
      - joint coordinates (as np.array of shape (N, 2))
      - mechanism type string (like "Watt1T2A1")
    """
    base = os.path.basename(filename).replace('.jpg', '').replace('.png', '')
    tokens = base.split()

    # Find where the mechanism name appears
    mech_idx = None
    for i, t in enumerate(tokens):
        if any(m in t.lower() for m in ["watt", "stephenson", "tchebychev", "hoeken", "klann", "jansen"]):
            mech_idx = i
            mech_type = t
            break

    if mech_idx is None:
        raise ValueError(f"Could not identify mechanism type in filename: {filename}")

    # Everything before mech_idx are joint coordinates
    coords_tokens = tokens[:mech_idx]
    coords = np.array(list(map(float, coords_tokens))).reshape(-1, 2)

    return coords, mech_type


def regenerate_mech_and_curve(joints, mech_type):
    payload = {
        "mechType": mech_type,
        "jointCoords": joints.tolist()
    }
    response = requests.post(API_ENDPOINT, headers=HEADERS, data=json.dumps(payload))

    if response.status_code != 200:
        raise RuntimeError(f"Simulation failed: {response.status_code} {response.text}")

    data = response.json()
    print("DEBUG - API response keys/type:", type(data))
    print("DEBUG - Example output:", list(data)[:3] if isinstance(data, list) else data.keys())
    return None



def detect_flat_via_pca(curve, window_size=20, flatness_threshold=0.001):
    """
    Detects flat portion of a curve using PCA curvature metric.
    """
    flat_indices = []
    for i in range(window_size, len(curve) - window_size):
        window = curve[i - window_size:i + window_size]
        pca = PCA(n_components=2)
        pca.fit(window)
        ratio = pca.explained_variance_ratio_[1] / pca.explained_variance_ratio_[0]
        if ratio < flatness_threshold:
            flat_indices.append(i)
    return {"flat_indices": flat_indices}


def fit_line_to_flat(curve, flat_indices):
    """
    Fits a best-fit line to the flat portion of the curve.
    """
    flat_pts = curve[flat_indices]
    coeffs = np.polyfit(flat_pts[:, 0], flat_pts[:, 1], 1)
    x_line = np.linspace(flat_pts[:, 0].min(), flat_pts[:, 0].max(), 100)
    y_line = np.polyval(coeffs, x_line)
    return x_line, y_line, coeffs


def plot_full_visualization(curve, joints, pca_output, title=None, save_path=None):
    """
    Plots the full visualization like the main script:
      - curve
      - red highlight for flat section
      - best-fit line
      - joint coordinates with indices
    """
    flat_indices = pca_output['flat_indices']

    plt.figure(figsize=(6, 6))
    plt.plot(curve[:, 0], curve[:, 1], 'k-', lw=1.5, label='Coupler Path')

    # Highlight flat section
    if len(flat_indices) > 0:
        plt.plot(curve[flat_indices, 0], curve[flat_indices, 1],
                 'r-', lw=2, label='Flat Portion')

        # Best-fit line
        x_line, y_line, _ = fit_line_to_flat(curve, flat_indices)
        plt.plot(x_line, y_line, 'b--', lw=1.5, label='Best-Fit Line')

    # Plot joints
    plt.scatter(joints[:, 0], joints[:, 1], c='g', s=40, label='Joints', zorder=3)
    for i, (x, y) in enumerate(joints):
        plt.text(x, y, str(i), fontsize=8, color='blue', ha='center', va='bottom')

    plt.title(title if title else "Mechanism Visualization")
    plt.axis('equal')
    plt.legend()
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


# ==================================================
# === Main function ===

def visualize_from_image(image_path, save_dir="./visualized"):
    joints, mech_type = parse_filename(image_path)
    print(f"🦿 Detected mechanism: {mech_type}")
    print(f"   → {joints.shape[0]} joints parsed from filename")

    # Re-simulate the curve
    print("🔁 Re-simulating mechanism via API...")
    curve = regenerate_mech_and_curve(joints, mech_type)

    # Detect flat section
    pca_output = detect_flat_via_pca(curve)

    # Plot and save visualization
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, os.path.basename(image_path))
    plot_full_visualization(curve, joints, pca_output, title=f"{mech_type} Visualization", save_path=save_path)

    print(f"✅ Saved visualization to {save_path}")


# ==================================================
# === Example usage ===
if __name__ == "__main__":
    image_path = "./outputs-6bar/walking_Watt1T2A1/0.644 9.467 8.386 4.924 -0.327 -3.451 0.752 -7.49 -2.203 -7.727 6.627 -3.455 4.48 -3.535 2.288 1.413 Watt1T2A1 -0.778 0.317 -4.048 -0.317 -0.778 6.42 0. 0. 1. .jpg"
    visualize_from_image(image_path)


🦿 Detected mechanism: Watt1T2A1
   → 8 joints parsed from filename
🔁 Re-simulating mechanism via API...
DEBUG - API response keys/type: <class 'list'>
DEBUG - Example output: []


TypeError: object of type 'NoneType' has no len()

In [10]:
image_path = "./outputs-6bar/walking_Watt1T2A1/0.644 9.467 8.386 4.924 -0.327 -3.451 0.752 -7.49 -2.203 -7.727 6.627 -3.455 4.48 -3.535 2.288 1.413 Watt1T2A1 -0.778 0.317 -4.048 -0.317 -0.778 6.42 0. 0. 1. .jpg"
joints, mech_type = parse_filename(image_path)
regenerate_mech_and_curve(joints, mech_type)


DEBUG - API response keys/type: <class 'list'>
DEBUG - Example output: []
